In [1]:
from multiprocessing import Pool, Process
import cv2
import numpy as np

In [2]:
import tensorflow as tf

2023-09-17 22:20:21.381736: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-17 22:20:21.979567: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-17 22:20:21.981576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-17 22:20:23.704619: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [4]:
if gpus:
    print('GPU(s) available')
    for gpu in gpus:
        print(gpu)
else:
    print('No GPU available, Using CPU.')

No GPU available, Using CPU.


In [5]:
import multiprocessing
import os

In [6]:
num_cpu_cores = multiprocessing.cpu_count()
print(f'Number of CPU cores: {num_cpu_cores}')

Number of CPU cores: 2


In [7]:
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

In [8]:
num_devices =2

In [27]:
file_path = '//home//hduser//Desktop//BANANAS'

In [28]:
img_height, img_width = 540, 960

In [29]:
def get_label(file_name):
    if file_name.startswith('g'):
        return 1
    elif file_name.startswith('Y'):
        return 2
    elif file_name.startswith('R'):
        return 3
    
        

In [30]:
def process_image(image_path):
    label = get_label(os.path.basename(image_path)[0])
    
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))
    
    img = img.astype('float32') /255.0
    
    return img, label

In [31]:
def train_model(data, labels):
    model = models.Sequential([
        layers.Conv2D(16, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(32, (3,3),activation = 'relu'),
        layers.Flatten(),
        layers.Dense(32, activation ='relu'),
        layers.Dense(3, activation ='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss = 'sparse_categorical_crossentropy',
                 metrics =['accuracy'])
    batch_size= 32
    
    
    history = model.fit(data, labels, epochs =10, validation_split=0.2, batch_size = batch_size)
    test_loss, test_acc = model.evaluate(date, labels, verbose = 2)
    print('Test accuracy:', test_acc)
    print('Test loss:', teest_loss)
    
    
    
    
    return model

In [32]:
if __name__ == '__main__':
    image_paths =[]
    for root, dirs, files in os.walk(file_path):
        for file in files:
            if file.endswith('.jpg'):
                image_paths.append(os.path.join(root, file))
                
                
    split_index = len(image_paths) //2
    image_paths_1, image_paths_2 = image_paths[:split_index], image_paths[split_index:]
    
    with Pool(2) as pool:
        data_labels_1 = pool.map(process_image, image_paths_1)
        data_labels_2 = pool.map(process_image, image_paths_2)
        
    data_1, labels_1 = zip(*data_labels_1)
    data_2, labels_2 = zip(*data_labels_2)
        
    data_1, labels_1 = np.array(data_1), np.array(labels_1)
    data_2, labels_2 = np.array(data_2), np.array(labels_2)
        
    process1 = Process(target = train_model, args =(data_1, labels_1))
    process2 = Process(target = train_model, args =(data_2, labels_2))
        
    process1.start()
    process2.start()
        
    process1.join()
    process2.join()
    

2023-09-17 22:58:29.169514: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 521662464 exceeds 10% of free system memory.
2023-09-17 22:58:29.178398: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 521662464 exceeds 10% of free system memory.
2023-09-17 22:58:32.462235: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 521662464 exceeds 10% of free system memory.
2023-09-17 22:58:32.468610: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 521662464 exceeds 10% of free system memory.
2023-09-17 22:58:33.839688: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 521662464 exceeds 10% of free system memory.
2023-09-17 22:58:33.839675: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 521662464 exceeds 10% of free system memory.


Epoch 1/10


2023-09-17 22:58:46.069907: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 521662464 exceeds 10% of free system memory.
2023-09-17 22:58:48.016277: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 521662464 exceeds 10% of free system memory.
2023-09-17 22:58:50.551044: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 3 which is outside the valid range of [0, 3).  Label values: 1 2 3 3 3 3 1 1 2 3 1 2
Process Process-31:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_4103/2450215831.py", line 17, in train_model
    history = model.fit(data, labels, epochs =10, validation_split=0.2, batch_size = batch_size)
  File "/home/hduser/.local/lib/py

In [ ]:
image_paths = []
for root, dirs, files in os.walk(file_path):
    for file in files:
        if file.endswith('.jpg'):
            image_paths.append(os.path.join(root, file))

In [ ]:
split_index = len(image_paths) // 2
image_paths_1, image_paths_2 = image_paths[:split_index], image_paths[split_index:]

In [ ]:
import cv2
from multiprocessing import Pool
with Pool(2) as pool:
    data_labels_1 = pool.map(process_image, image_paths_1)
    data_labels_2 = pool.map(process_image, image_paths_2)

In [ ]:
data_1, labels_1 = zip(*data_labels_1)
data_2, labels_2 = zip(*data_labels_2)

In [ ]:
import numpy as np
data_1, labels_1 = np.array(data_1), np.array(labels_1)

In [ ]:
import numpy as np
data_2, labels_2 = np.array(data_2), np.array(labels_2)

In [ ]:
data_labels_1

In [ ]:
labels_1

In [ ]:
def train_model()

In [ ]:
import cv2

In [ ]:
data = []
labels = []

for root, dirs, files in os.walk(file_path):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            label = get_label(file[0])
            
            
            img = cv2.imread(image_path)
            img = cv2.resize(img, (img_width, img_height))
            
            
            data.append(img)
            labels.append(label)

In [ ]:
data

In [ ]:
import numpy as np

labels = np.array(labels)
labels

In [ ]:

data = np.array(data, dtype = 'float32')


In [ ]:
target_hight, target_width = 540, 960

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale = 1.0/255)

In [ ]:
x_paths = []
y = []

for class_name in os.listdir(file_path):
    class_path = os.path.join(file_path, class_name)
    
    if os.path.isdir(class_path):
        if class_name.startswith('G'):
            label = 0
        elif class_name.startswith('Y'):
            label = 1
        elif class_name.startswith('R'):
            label = 2
        else:
            continue
            
        
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            if image_name.endswith('.jpg'):
                image = load_and_preprocess_image(image_path)
                x_paths.append(image_path)
                y.append(label)

In [ ]:
def load_and_preprocess_image(file_path):
    image = cv2.imread(file_path)
    
    image = cv2.resize(image, (960,540))
    
    image = image.astype(np.float32)/255.0
    
    return image

In [ ]:
image_paths = []
labels = []

for class_name in os.listdir(file_path):
    class_path = os.path.join(file_path, class_name)
    
    if os.path.isdir(class_path):
        if class_name.startswith('G'):
            label = 0
        elif class_name.startswith('Y'):
            label = 1
        elif class_name.startswith('R'):
            label = 2
        else:
            continue
            
        
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            if image_name.endswith('.jpg'):
                image = load_and_preprocess_image(image_path)
                image_paths.append(image_path)
                labels.append(label)
                

In [ ]:
X = []

for image_path in image_paths:
    image = tf.keras.preprocessing.image.load_img(image_path)
    image = tf.keras.preprocessing.image.img_to_array(image)
   # image = datagen.standarize(image)
    X.append(image)

In [ ]:
X

In [ ]:
X = tf.convert_to_tensor(X)
y = tf.convert_to_tensor(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size =0.3, random_state=42)


In [ ]:
X_test

In [ ]:
X_train =[]
for 

In [ ]:
y_test

In [ ]:
num_devices =2
num_samples = len(X_train)

In [ ]:
batch_size_per_device = num_samples // num_devices
data_subsets = [(X_train[i:i+batch_size_per_device], y_train[i:i+batch_size_per_device]) for i in range(0, num_samples, batch_size_per_device)]

In [ ]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation ='relu', input_shape = (32,32,3)),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(64, activation ='relu'),
        layers.Dense(10, activation ='softmax')
    ])
    model.compile(optimizer = 'adam',
                 loss ='sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
    return model

In [ ]:
models_list =[create_model() for _ in range(num_devices)]

In [ ]:
optimizer_list = tf.keras.optimizers.SGD(learning_rate =0.01)

In [ ]:
num_epochs =10
for epoch in range(num_epochs):
    for i in range(num_devices):
        x_batch, y_batch = data_subsets[i]
        
        with tf.GradientTape() as tape:
            predictions = models_list[i](x_batch)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, predictions)
            
        gradients = tape.gradient(loss, models_list[i].trainable_variables)
        optimizer.apply_gradients(zip(gradients,  odels_list[i].trainable_variables))
        
        
    for i in range(1, num_devices):
        models_list[i].set_weights(models_list[0].get_weights())
        
final_model = model_list[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import EarlyStopping
import time
import requests

In [ ]:
print(labels)

In [ ]:
X_train

In [ ]:
start_time = time.time()

model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(540,960,3)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation ='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
             loss='sparse_cetegorical_crossentropy',
             metrics=['accuracy'])

X_train_paths = X_train
X_train_pixels = np.array([load_and_preprocess_image(image_path) for image_path in X_train_paths])
y_train = np.array(y_train)

X_val_paths = X_val
X_val_pixels = np.array([load_and_preprocess_image(image_path) for image_path in X_val_paths])
y_val = np.array(y_val)

batch_size = 16
epochs = 10

history = model.fit(
    X_train_pixels,
    y_train,
    epochs = epochs,
    batch_size = batch_size,
    validation_data = (X_val_pixels, y_val)
)

end_time = time.time()

elapsed_time = end_time - start_time

print(f"Time taken: {elapsed_time} seconds")
